<a href="https://colab.research.google.com/github/villafue/Capstone_2_MovieLens/blob/main/Notebook/9_Hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9.

<a name="top"></a>
## Hybrid Recommendation Systems

### Table of Contents

Note: The internal links work in Google Colab.

1. **[Preface](https://colab.research.google.com/github/villafue/Capstone_2_MovieLens/blob/main/MovieLens.ipynb#preface)**
2. **[Introduction](https://colab.research.google.com/github/villafue/Capstone_2_MovieLens/blob/main/MovieLens.ipynb#introduction)**
3. **[Exploratory Data Analysis](https://colab.research.google.com/github/villafue/Capstone_2_MovieLens/blob/main/Notebook/3_Exploratory_Data_Analysis.ipynb.ipynb#eda)**
4. **[Framework](https://colab.research.google.com/github/villafue/Capstone_2_MovieLens/blob/main/Notebook/4_Framework.ipynb#framework)**
5. **[Content Based Recommenders](https://colab.research.google.com/github/villafue/Capstone_2_MovieLens/blob/main/Notebook/5_Content_Based_Recommenders.ipynb#content)**
6. **[Collaborative Based Recommenders](https://colab.research.google.com/github/villafue/Capstone_2_MovieLens/blob/main/Notebook/6_Collaborative_Based_Recommenders.ipynb#collaborative)**
7. **[Matrix Factorization Methods](https://colab.research.google.com/github/villafue/Capstone_2_MovieLens/blob/main/Notebook/7_Matrix_Factorization_Methods.ipynb#matrix)**
8. **[Deep Learning Recommenders](#deep_learning)**
9. **[Hybrid Recommenders](#hybrid)**
    - 9.1 - [Introduction](#introduction)
    - 9.2 - [HybridAlgorithm.py](#hybrid_algo)
    - 9.3 - [Import Files](#import)
    - 9.4 - [Models](#models)
      - 9.4.1 - [RBM | ContentKNN = 50/50](#941)
      - 9.4.2 - [ContentKNN (Reverse=True) | Random = 50/50](#942)
      - 9.4.3 - [ContentKNN (Reverse=False) | Random = 50/50](#943)
      - 9.4.4 - [RBM | SVD++Untuned | SVDTuned | Random = 30/30/30/10](#944)
      - 9.4.5 - [RBM | SVD++Untuned | SVDTuned | ContentKNN | AutoRec = 20/20/20/20/20](#945)
      - 9.4.6 - [RBM | SVD++Untuned | SVDTuned = 33/34/33](#946)
      - 9.4.7 - [SVD++Untuned | SVDTuned = 50/50](#947)
      - 9.4.8 - [SVD++Untuned | SVDTuned | Random = 33/34/33](#948)
      - 9.4.8 - [SVD++Untuned | SVDTuned | Random = 33/34/33](#948)
    - 8.5 - [Best Model](#best_model)

***

<a name="introduction"></a>
### 9.1 - Introduction

In the real world, there's no need to choose a single algorithm for a recommender system. Each algorithm has its own strengths and weaknesses and it's possible to create a better system by combining many models together in a hybrid system. We touched on it earlier, but these hybrid or ensemble approaches can make a real difference. In fact, winner of the [Netflix prize](https://en.wikipedia.org/wiki/Recommender_system#Hybrid_recommender_systems) won by creating an ensemble of 107 different algorithms! 

In this section, I will introduce Frank's framework for Hybrid models, create and try different hybrid recommender systems, and choose the best one.
 


***

**[Back to Top](#top)**

***

<a name="hybrid_algo"></a>
### 9.2 - HybridAlgorithm.py

`HybridAlgorithm.py` generates rating predictions from many recommenders in parallel and averages their scores together before ranking them. This is done through the `__init__` **(line 12)** function as takes in a list of algorithms and an associated list of weights **(line 15)** assigned to each one. The `estimate` **(line 25)** function combines the resulting predictions in a weighted average **(line 31 - 32)**, which makes up the final rating estimate **(line 34)**.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri May  4 13:08:25 2018

@author: Frank
"""

from surprise import AlgoBase

class HybridAlgorithm(AlgoBase):

    def __init__(self, algorithms, weights, sim_options={}):
        AlgoBase.__init__(self)
        self.algorithms = algorithms
        self.weights = weights

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        
        for algorithm in self.algorithms:
            algorithm.fit(trainset)
                
        return self

    def estimate(self, u, i):
        
        sumScores = 0
        sumWeights = 0
        
        for idx in range(len(self.algorithms)):
            sumScores += self.algorithms[idx].estimate(u, i) * self.weights[idx]
            sumWeights += self.weights[idx]
            
        return sumScores / sumWeights

    

***

**[Back to Top](#top)**

***

<a name="import"></a>
### 9.3 - Import Files

In [ ]:
import os
os.mkdir('/content/hybrid')
print('Folder created!')
os.chdir('/content/hybrid')
print('Files are in this folder!')

Folder created!
Files are in this folder!


In [ ]:
pip install scikit-surprise

     |████████████████████████████████| 11.8MB 10.4MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617546 sha256=a782c72ddbd28c70bebcff54212f64447a407c5acb2f5b5fc1e03aa3f93d4355
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
print("Loading Architecture.")
!python "MovieLens.py"
print('1 of 5: Done')
!python "RecommenderMetrics.py"
print('2 of 5: Done')
!python "EvaluationData.py"
print('3 of 5: Done')
!python "EvaluatedAlgorithm.py"
print('4 of 5: Done')
!python "Evaluator.py"
print('5 of 5: Core Framework Loaded!')

Loading Architecture.
1 of 5: Done
2 of 5: Done
3 of 5: Done
4 of 5: Done
5 of 5: Core Framework Loaded!


I loaded all the algorithms I would use in the different Hybrid models. The files are located in this folder [here](https://github.com/villafue/Capstone_2_MovieLens/tree/main/Python%20Scripts/Hybrid_Final). Generally, each of these algorithms were the exact ones I used in the Hybrid models. However, sometimes I made adjustment which, if it was the case, I specified those adjustments with the model.

In [ ]:
print('Loading Model Algorithms')
print('\n', '-' * 136)
!python "RBM.py"
!python "RBMAlgorithm.py"
!python "AutoRec.py"
!python "AutoRecAlgorithm.py"
print('\n', '-' * 136) 
print('1 of 4: Loaded RBM')
print('2 of 4: Loaded AutoRec')
!python "ContentKNNAlgorithm.py"
print('3 of 4: Loaded Content-based KNN')
!python "HybridAlgorithm.py"
print('4 of 4: Loaded Hybrid Model')
print('All Models Loaded!')

Loading Model Algorithms

 ----------------------------------------------------------------------------------------------------------------------------------------
2021-03-24 17:48:46.316112: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-24 17:48:48.905102: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-24 17:48:51.054009: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-24 17:48:53.762753: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

 ----------------------------------------------------------------------------------------------------------------------------------------
1 of 4: Loaded RBM
2 of 4: Loaded AutoRec
3 of 4: Loaded Content-based KNN
4 of 4: Loaded Hybrid Model
All Models

***

**[Back to Top](#top)**

***

<a name="models"></a>
### 9.4 - Models

Each of these hybrid recommenders are titled by their component estimators and the weights associated with them. For example, the first model below, "**RBM | ContentKNN = 50/50**" is the hybrid model where both RBM and ContentKNN algorithms are evenly weighted in the ensemble.

Furthermore, Frank said that sometimes, "Developing recommender systems sometimes feels like more of an art than a science, and choosing how to combine different algorithms can feel like choosing different colors in a painting that complement each other." This definitely holds true in this section. I tried many combinations of recommender systems without having much intuition into how the ensembles would turn out. I'm thankful for his framework, however, as the different hybrid models were easy to code. 

Please note that most of these models took well over 4 hours to run, and some over 12. This is because I printed out all the metrics for all the base recommenders, and then ultimately the hybrid algorithm itself.



***

<a name="941"></a>
#### 9.4.1 - RBM | ContentKNN = 50/50

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from RBMAlgorithm import RBMAlgorithm
from ContentKNNAlgorithm import ContentKNNAlgorithm
from HybridAlgorithm import HybridAlgorithm
from Evaluator import Evaluator

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

#Tuned RBM
RBM = RBMAlgorithm(epochs=40, learningRate=0.02)
#Content
ContentKNN = ContentKNNAlgorithm()

#Combine them
Hybrid = HybridAlgorithm([RBM, ContentKNN], [0.5, 0.5])

evaluator.AddAlgorithm(RBM, "RBM")
evaluator.AddAlgorithm(ContentKNN, "ContentKNN")
evaluator.AddAlgorithm(Hybrid, "Hybrid")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)


Instructions for updating:
non-resource variables are not supported in the long term
Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  RBM ...
Evaluating accuracy...
Blocked  sex, lies, and videotape (1989)
Blocked  sexy beast (2000)
Blocked  sex and the city (2008)
Blocked  i am a sex addict (2005)
Blocked  sex, drugs & taxation (2013)
Blocked  sex, drugs & taxation (2013)
Blocked  everything you always wanted to know about sex * but were afraid to ask (1972)
Blocked  sex and lucia (lucã­a y el sexo) (2001)
Blocked  drugstore cowboy (1989)
Blocked  sex drive (2008)
Blocked  the opposite sex (2014)
Blocked  sex tape (2014)
Blocked  opposite of sex, the (1998)
Blocked  loss of sexual innocence, the (1999)
Blocked  sex and the city 2 (2010)
Blocked  sex ed (2014)
Blocked  love and other drugs (2010)
Blocked  better than sex (20

Although the `RMSE` is not too bad, all the hit rate metrics were awful. 

```
Algorithm  RMSE       MAE        HR         cHR        ARHR       Coverage   Diversity  Novelty   
RBM        1.1563     0.9586     0.0000     0.0000     0.0000     0.0000     0.2062     1410.4948 
ContentKNN 0.9055     0.6983     0.0000     0.0000     0.0000     0.9213     0.6284     4911.3169 
Hybrid     0.9640     0.7788     0.0000     0.0000     0.0000     0.4311     0.2442     2142.3238 

Legend:

RMSE:      Root Mean Squared Error. Lower values mean better accuracy.
MAE:       Mean Absolute Error. Lower values mean better accuracy.
HR:        Hit Rate; how often we are able to recommend a left-out rating. Higher is better.
cHR:       Cumulative Hit Rate; hit rate, confined to ratings above a certain threshold. Higher is better.
ARHR:      Average Reciprocal Hit Rank - Hit rate that takes the ranking into account. Higher is better.
Coverage:  Ratio of users for whom recommendations above a certain threshold exist. Higher is better.
Diversity: 1-S, where S is the average similarity score between every possible pair of recommendations
           for a given user. Higher means more diverse.
Novelty:   Average popularity rank of recommended items. Higher means more novel.
```

Below, none of the recommendations make me excited. I don't like this model.

```
Using recommender  RBM
We recommend:
Mr. Smith Goes to Washington (1939) 3.3610125
Exit Through the Gift Shop (2010) 3.3610125
Crumb (1994) 3.3610125
Crimes and Misdemeanors (1989) 3.3610125
All About My Mother (Todo sobre mi madre) (1999) 3.3610125
Despicable Me 2 (2013) 3.3610125
About Time (2013) 3.3610125
Creature Comforts (1989) 3.3610125
Lives of Others, The (Das leben der Anderen) (2006) 3.3610125
Eastern Promises (2007) 3.3610125

Using recommender  ContentKNN
We recommend:
Psycho (1960) 5
Troll 2 (1990) 5
Candyman: Farewell to the Flesh (1995) 5
2 Days in the Valley (1996) 5
Sherlock - A Study in Pink (2010) 5
Sound of Music, The (1965) 5
South Pacific (1958) 5
'Tis the Season for Love (2015) 5
Fog of War: Eleven Lessons from the Life of Robert S. McNamara, The (2003) 5
Message in a Bottle (1999) 5

Using recommender  Hybrid
We recommend:
Fog of War: Eleven Lessons from the Life of Robert S. McNamara, The (2003) 4.172626852989197
King and I, The (1956) 4.1693185253172595
Mr. Smith Goes to Washington (1939) 4.168823913570082
Grave of the Fireflies (Hotaru no haka) (1988) 4.167955289595813
Paths of Glory (1957) 4.165096656173512
All About My Mother (Todo sobre mi madre) (1999) 4.164990908563042
Downfall (Untergang, Der) (2004) 4.163641385844867
On the Waterfront (1954) 4.162392735344899
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950) 4.161439029696532
Rashomon (RashÃ´mon) (1950) 4.161337516417975
```

***

**[Back to Top](#top)**

***

<a name="942"></a>
#### 9.4.2 - ContentKNN (Reverse=True) | Random = 50/50 * 

I wanted to try `ContentKNN` because it does not have a data sparsity issue. It only depends on the content of the movies itself for the algorithm. This version of `ContentKNN` only uses dates of the movies, and not genre, for its algorithm. Furthermore, I often preferred the `Random` model's movie recommendations over that of some of my models. I also thought adding `Random` in my hybrid models would help give users new and fresh movies they otherwise would never watch.

However, there is an issue using `ContentKNN` that I need to address, and it's the same one from section [5.4.3](https://colab.research.google.com/github/villafue/Capstone_2_MovieLens/blob/main/Notebook/5_Content_Based_Recommenders.ipynb#date_default) where a line of code gives me older movie recommendations. I prefer to see more recent and more popular movies and I believe the default setting tends to show older movies. 


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Mar 18 17:32:44 2021

@author: Maribel
"""

# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from ContentKNNAlgorithm import ContentKNNAlgorithm
from HybridAlgorithm import HybridAlgorithm
from Evaluator import Evaluator
from surprise import NormalPredictor

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# Just make random recommendations
Random = NormalPredictor()
evaluator.AddAlgorithm(Random, "Random")

#Content
ContentKNN = ContentKNNAlgorithm()

#Combine them
Hybrid = HybridAlgorithm([Random, ContentKNN], [0.5, 0.5])

evaluator.AddAlgorithm(ContentKNN, "ContentKNN")
evaluator.AddAlgorithm(Hybrid, "Hybrid")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)


Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  Random ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  ContentKNN ...
Evaluating accuracy...
Computing content-based similarity matrix...
0  of  8805
100  of  8805
200  of  8805
300  of  8805
400  of  8805
500  of  8805
600  of  8805
700  of  8805
800  of  8805
900  of  8805
1000  of  8805
1100  of  8805
1200  of  8805
1300  of  8805
1400  of  8805
1500  of  8805
1600  of  8805
1700  of  8805
1800  of  8805
1900  of  8805
2000  of  8805
2100  of  8805
2200  of  8805
2300  of  8805
2400  of  8805
2500  of  8805
2600  of  880

The `RMSE` is not bad for the Hybrid model although the `Novelty` score is pretty high. 
```
Algorithm  RMSE       MAE        HR         cHR        ARHR       Coverage   Diversity  Novelty   
Random     1.4139     1.1282     0.0180     0.0180     0.0072     1.0000     0.0522     854.7062  
ContentKNN 0.9322     0.7235     0.0082     0.0082     0.0017     0.5492     0.3002     2672.1657 
Hybrid     1.0870     0.8592     0.0049     0.0049     0.0014     1.0000     0.4101     2675.6103 

Legend:

RMSE:      Root Mean Squared Error. Lower values mean better accuracy.
MAE:       Mean Absolute Error. Lower values mean better accuracy.
HR:        Hit Rate; how often we are able to recommend a left-out rating. Higher is better.
cHR:       Cumulative Hit Rate; hit rate, confined to ratings above a certain threshold. Higher is better.
ARHR:      Average Reciprocal Hit Rank - Hit rate that takes the ranking into account. Higher is better.
Coverage:  Ratio of users for whom recommendations above a certain threshold exist. Higher is better.
Diversity: 1-S, where S is the average similarity score between every possible pair of recommendations
           for a given user. Higher means more diverse.
Novelty:   Average popularity rank of recommended items. Higher means more novel.

```

I was worried about the dates being too old but I actually kind of like the Hybrid recommendations. Maybe it's due to adding the `Random` algorithm.

```
Using recommender  Random
We recommend:
Tommy Boy (1995) 5
Jungle Book, The (1994) 5
Wizard of Oz, The (1939) 5
Citizen Kane (1941) 5
Mr. Smith Goes to Washington (1939) 5
Escape to Witch Mountain (1975) 5
Henry V (1989) 5
Last of the Mohicans, The (1992) 5
Conan the Barbarian (1982) 5
Game, The (1997) 5

Using recommender  ContentKNN
We recommend:
Snow White and the Seven Dwarfs (1937) 4.906496171944079
King Solomon's Mines (1937) 4.906496171944079
Gold Rush, The (1925) 4.906496171944079
Battleship Potemkin (1925) 4.906496171944079
Star Is Born, A (1937) 4.906496171944079
Shall We Dance (1937) 4.906496171944079
Ben-Hur: A Tale of the Christ (1925) 4.906496171944079
Grand Illusion (La grande illusion) (1937) 4.906496171944079
Stage Door (1937) 4.906496171944079
Awful Truth, The (1937) 4.906496171944079

Using recommender  Hybrid
We recommend:
Grumpier Old Men (1995) 5
Space Jam (1996) 5
Swingers (1996) 5
Monty Python's Life of Brian (1979) 5
Platoon (1986) 5
Clockwork Orange, A (1971) 5
Alien (1979) 5
Indiana Jones and the Last Crusade (1989) 5
Starship Troopers (1997) 5
Goonies, The (1985) 5
```

***

**[Back to Top](#top)**

***

<a name="943"></a>
#### 9.4.3 - ContentKNN (Reverse=False) | Random = 50/**50**

This is the same model as above except I switched a few lines of code to sort the top 10 recommendations in reverse. This solution came from section [5.4.4](https://colab.research.google.com/github/villafue/Capstone_2_MovieLens/blob/main/Notebook/5_Content_Based_Recommenders.ipynb#date_reverse)

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Mar 18 17:32:44 2021

@author: Maribel
"""

# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from ContentKNNAlgorithm import ContentKNNAlgorithm
from HybridAlgorithm import HybridAlgorithm
from Evaluator import Evaluator
from surprise import NormalPredictor

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# Just make random recommendations
Random = NormalPredictor()
evaluator.AddAlgorithm(Random, "Random")

#Content
ContentKNN = ContentKNNAlgorithm()

#Combine them
Hybrid = HybridAlgorithm([Random, ContentKNN], [0.5, 0.5])

evaluator.AddAlgorithm(ContentKNN, "ContentKNN")
evaluator.AddAlgorithm(Hybrid, "Hybrid")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)


Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  Random ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  ContentKNN ...
Evaluating accuracy...
Computing content-based similarity matrix...
0  of  8805
100  of  8805
200  of  8805
300  of  8805
400  of  8805
500  of  8805
600  of  8805
700  of  8805
800  of  8805
900  of  8805
1000  of  8805
1100  of  8805
1200  of  8805
1300  of  8805
1400  of  8805
1500  of  8805
1600  of  8805
1700  of  8805
1800  of  8805
1900  of  8805
2000  of  8805
2100  of  8805
2200  of  8805
2300  of  8805
2400  of  8805
2500  of  8805
2600  of  880

This hybrid model's metrics are very similar to my last model.

```
Algorithm  RMSE       MAE        HR         cHR        ARHR       Coverage   Diversity  Novelty   
Random     1.4139     1.1282     0.0180     0.0180     0.0072     1.0000     0.0522     854.7062  
ContentKNN 0.9322     0.7235     0.0082     0.0082     0.0017     0.5492     0.3002     2672.1657 
Hybrid     1.0870     0.8592     0.0049     0.0049     0.0014     1.0000     0.4101     2675.6103 

Legend:

RMSE:      Root Mean Squared Error. Lower values mean better accuracy.
MAE:       Mean Absolute Error. Lower values mean better accuracy.
HR:        Hit Rate; how often we are able to recommend a left-out rating. Higher is better.
cHR:       Cumulative Hit Rate; hit rate, confined to ratings above a certain threshold. Higher is better.
ARHR:      Average Reciprocal Hit Rank - Hit rate that takes the ranking into account. Higher is better.
Coverage:  Ratio of users for whom recommendations above a certain threshold exist. Higher is better.
Diversity: 1-S, where S is the average similarity score between every possible pair of recommendations
           for a given user. Higher means more diverse.
Novelty:   Average popularity rank of recommended items. Higher means more novel.
```

Interestingly, the hybrid recommendations are worse. I do not recommend this model.
```
Using recommender  Random
We recommend:
Waiting for Guffman (1996) 1
Smoke Signals (1998) 1
Bowfinger (1999) 1
Dunston Checks In (1996) 1
Angus (1995) 1
First Kid (1996) 1
Aladdin and the King of Thieves (1996) 1
Road to Perdition (2002) 1
Boondock Saints, The (2000) 1
Girl Who Kicked the Hornet's Nest, The (Luftslottet som sprÃ¤ngdes) (2009) 1

Using recommender  ContentKNN
We recommend:
Ant-Man and the Wasp (2018) 4.7450497998793395
The Darkest Minds (2018) 4.7450497998793395
Annihilation (2018) 4.7450497998793395
Game Night (2018) 4.7450497998793395
Tomb Raider (2018) 4.7450497998793395
Alpha (2018) 4.7450497998793395
Solo: A Star Wars Story (2018) 4.7450497998793395
Fred Armisen: Standup for Drummers (2018) 4.7450497998793395
Tom Segura: Disgraceful (2018) 4.7450497998793395
When We First Met (2018) 4.7450497998793395

Using recommender  Hybrid
We recommend:
Guru, The (2002) 2.2533277089928374
Sweetie (1989) 2.2567271670320483
Shining, The (1980) 2.345221848464202
Bad Santa 2 (2016) 2.3873090593255797
Faraway, So Close (In weiter Ferne, so nah!) (1993) 2.3965209235043896
Onion Movie, The (2008) 2.4049091160585014
Honeymoon in Vegas (1992) 2.407767729693857
Envy (2004) 2.4292778655522076
Pulse (2006) 2.493121051418905
Gregory's Girl (1981) 2.5000196675682265
```

***

**[Back to Top](#top)**

***

<a name="944"></a>
#### 9.4.4 - RBM | SVD++Untuned | SVDTuned | Random = 30/30/30/10

I wondered how much `Random` is affecting my hybrid models by giving it 50% weight. In this one, I lowered it to 10%. I also used both `SVD++Untuned` and `SVDTuned` algorithms because both of them had great movie recommendations. I used those with `RBM` because I thought their algorithms could help each other. 

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from RBMAlgorithm import RBMAlgorithm
from ContentKNNAlgorithm import ContentKNNAlgorithm
from HybridAlgorithm import HybridAlgorithm
from surprise import SVD, SVDpp
from surprise import NormalPredictor
from Evaluator import Evaluator

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# RBM
RBM = RBMAlgorithm(epochs=40, hiddenDim=40, learningRate=0.2)

# SVD++ Untuned
SVDPlus = SVDpp()

# SVD Tuned
SVDTuned = SVD(n_epochs=11, lr_all=0.034, n_factors=90, reg_all=0.06)

Random = NormalPredictor()

#Content
#ContentKNN = ContentKNNAlgorithm()

#Combine them
Hybrid = HybridAlgorithm([RBM, SVDPlus, SVDTuned, Random], [0.3, 0.3, 0.3, 0.1])

evaluator.AddAlgorithm(RBM, "RBM")
evaluator.AddAlgorithm(SVDPlus, "SVD++Untuned")
evaluator.AddAlgorithm(SVDTuned, "SVD Tuned")
evaluator.AddAlgorithm(Random, "Random")
#evaluator.AddAlgorithm(ContentKNN, "ContentKNN")
evaluator.AddAlgorithm(Hybrid, "Hybrid")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)

Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  RBM ...
Evaluating accuracy...
Trained epoch  0
Trained epoch  1
Trained epoch  2
Trained epoch  3
Trained epoch  4
Trained epoch  5
Trained epoch  6
Trained epoch  7
Trained epoch  8
Trained epoch  9
Trained epoch  10
Trained epoch  11
Trained epoch  12
Trained epoch  13
Trained epoch  14
Trained epoch  15
Trained epoch  16
Trained epoch  17
Trained epoch  18
Trained epoch  19
Trained epoch  20
Trained epoch  21
Trained epoch  22
Trained epoch  23
Trained epoch  24
Trained epoch  25
Trained epoch  26
Trained epoch  27
Trained epoch  28
Trained epoch  29
Trained epoch  30
Trained epoch  31
Trained epoch  32
Trained epoch  33
Trained epoch  34
Trained epoch  35
Trained epoch  36
Trained epoch  37
Trained epoch  38
Trained epoch  39
Processing user  0
Processing user  50
Process

The `RMSE` is not too high and I thank both SVD modelds for that. Novelty isn't too bad either.

```
Algorithm    RMSE       MAE        HR         cHR        ARHR       Coverage   Diversity  Novelty   
RBM          1.2854     1.0926     0.0213     0.0213     0.0073     0.0000     0.0506     938.5557  
SVD++Untuned 0.8656     0.6650     0.0213     0.0213     0.0114     0.9164     0.0959     924.7326  
SVD Tuned    0.8686     0.6680     0.0180     0.0180     0.0080     0.9607     0.3015     2057.0823 
Random       1.4192     1.1340     0.0115     0.0115     0.0039     1.0000     0.0502     852.1002  
Hybrid       0.9304     0.7475     0.0197     0.0197     0.0069     0.6115     0.1956     1613.4659 

Legend:

RMSE:      Root Mean Squared Error. Lower values mean better accuracy.
MAE:       Mean Absolute Error. Lower values mean better accuracy.
HR:        Hit Rate; how often we are able to recommend a left-out rating. Higher is better.
cHR:       Cumulative Hit Rate; hit rate, confined to ratings above a certain threshold. Higher is better.
ARHR:      Average Reciprocal Hit Rank - Hit rate that takes the ranking into account. Higher is better.
Coverage:  Ratio of users for whom recommendations above a certain threshold exist. Higher is better.
Diversity: 1-S, where S is the average similarity score between every possible pair of recommendations
           for a given user. Higher means more diverse.
Novelty:   Average popularity rank of recommended items. Higher means more novel.
```

Looking below, the movie recommendations are not exciting despite the decent metrics. I do not recommend this model.

```
Using recommender  RBM
We recommend:
Crucible, The (1996) 2.8537362
Lawrence of Arabia (1962) 2.8520052
Back to the Future (1985) 2.8502877
Fugitive, The (1993) 2.8500934
Aliens (1986) 2.849308
True Lies (1994) 2.8491013
Speed (1994) 2.8490562
Heat (1995) 2.8490326
Die Hard 2 (1990) 2.8489242
Quiz Show (1994) 2.8486722

Using recommender  SVD++Untuned
We recommend:
Braveheart (1995) 5
Goodfellas (1990) 5
Shining, The (1980) 5
American History X (1998) 5
Shawshank Redemption, The (1994) 5
Good Will Hunting (1997) 5
Departed, The (2006) 5
In the Name of the Father (1993) 5
Philadelphia Story, The (1940) 5
Singin' in the Rain (1952) 5

Using recommender  SVD Tuned
We recommend:
Usual Suspects, The (1995) 5
Pulp Fiction (1994) 5
Star Wars: Episode V - The Empire Strikes Back (1980) 5
Fight Club (1999) 5
Shawshank Redemption, The (1994) 5
Departed, The (2006) 5
Philadelphia Story, The (1940) 5
Life Is Beautiful (La Vita Ã¨ bella) (1997) 5
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) 5
Godfather, The (1972) 5

Using recommender  Random
We recommend:
Toy Story (1995) 5
Desperado (1995) 5
Dazed and Confused (1993) 5
Tombstone (1993) 5
Independence Day (a.k.a. ID4) (1996) 5
Three Caballeros, The (1945) 5
Pete's Dragon (1977) 5
Monty Python's Life of Brian (1979) 5
Kiss the Girls (1997) 5
Newton Boys, The (1998) 5

Using recommender  Hybrid
We recommend:
Paperman (2012) 4.4813628621609745
Shawshank Redemption, The (1994) 4.480401631452825
Celebration, The (Festen) (1998) 4.434521828207211
Streetcar Named Desire, A (1951) 4.410752387316927
Creature Comforts (1989) 4.400548585334615
Howl's Moving Castle (Hauru no ugoku shiro) (2004) 4.386544185994829
Shining, The (1980) 4.382935959652201
Thank You for Smoking (2006) 4.358162979739672
General, The (1926) 4.350943784983842
Intouchables (2011) 4.341596594170237
```

***

**[Back to Top](#top)**

***

<a name="945"></a>
#### 9.4.5 - RBM | SVD++Untuned | SVDTuned | ContentKNN | AutoRec = 20/20/20/20/20

I decided to try both deep learning models, both SVD models, and my content-based model. My reasoning was that maybe all these different algorithms, weighted equally, would give me great recommendations.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from RBMAlgorithm import RBMAlgorithm
from ContentKNNAlgorithm import ContentKNNAlgorithm
from AutoRecAlgorithm import AutoRecAlgorithm
from HybridAlgorithm import HybridAlgorithm
from surprise import SVD, SVDpp
from surprise import NormalPredictor
from Evaluator import Evaluator

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# RBM
RBM = RBMAlgorithm(epochs=40, hiddenDim=40, learningRate=0.2)

# SVD++ Untuned
SVDPlus = SVDpp()

# SVD Tuned
SVDTuned = SVD(n_epochs=11, lr_all=0.034, n_factors=90, reg_all=0.06)

Random = NormalPredictor()

#Content
ContentKNN = ContentKNNAlgorithm()

#AutoRec
#{'hiddenDim': 200, 'learningRate': 0.1}
AutoRec = AutoRecAlgorithm(hiddenDim=200, learningRate=0.1)

#Combine them
Hybrid = HybridAlgorithm([RBM, SVDPlus, SVDTuned, ContentKNN, AutoRec], [0.2, 0.2, 0.2, 0.2, 0.2])

evaluator.AddAlgorithm(RBM, "RBM")
evaluator.AddAlgorithm(SVDPlus, "SVD++Untuned")
evaluator.AddAlgorithm(SVDTuned, "SVD Tuned")
evaluator.AddAlgorithm(ContentKNN, "ContentKNN")
evaluator.AddAlgorithm(AutoRec, "Auto Rec")
#evaluator.AddAlgorithm(Random, "Random")
evaluator.AddAlgorithm(Hybrid, "Hybrid")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)

Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  RBM ...
Evaluating accuracy...
Trained epoch  0
Trained epoch  1
Trained epoch  2
Trained epoch  3
Trained epoch  4
Trained epoch  5
Trained epoch  6
Trained epoch  7
Trained epoch  8
Trained epoch  9
Trained epoch  10
Trained epoch  11
Trained epoch  12
Trained epoch  13
Trained epoch  14
Trained epoch  15
Trained epoch  16
Trained epoch  17
Trained epoch  18
Trained epoch  19
Trained epoch  20
Trained epoch  21
Trained epoch  22
Trained epoch  23
Trained epoch  24
Trained epoch  25
Trained epoch  26
Trained epoch  27
Trained epoch  28
Trained epoch  29
Trained epoch  30
Trained epoch  31
Trained epoch  32
Trained epoch  33
Trained epoch  34
Trained epoch  35
Trained epoch  36
Trained epoch  37
Trained epoch  38
Trained epoch  39
Processing user  0
Processing user  50
Process

The metrics are not too bad and the `RMSE` is pretty low due to the non-neural network algorithms.

```
Algorithm    RMSE       MAE        HR         cHR        ARHR       Coverage   Diversity  Novelty   
RBM          1.2855     1.0927     0.0213     0.0213     0.0063     0.0000     0.0514     946.6134  
SVD++Untuned 0.8656     0.6650     0.0213     0.0213     0.0114     0.9164     0.0959     924.7326  
SVD Tuned    0.8686     0.6680     0.0180     0.0180     0.0080     0.9607     0.3015     2057.0823 
ContentKNN   0.9025     0.6976     0.0000     0.0000     0.0000     0.9213     0.6284     4911.3169 
Auto Rec     1.9125     1.5527     0.0049     0.0049     0.0025     1.0000     0.1073     1056.2369 
Hybrid       1.0254     0.8203     0.0131     0.0131     0.0063     0.7672     0.2031     1802.6167 

Legend:

RMSE:      Root Mean Squared Error. Lower values mean better accuracy.
MAE:       Mean Absolute Error. Lower values mean better accuracy.
HR:        Hit Rate; how often we are able to recommend a left-out rating. Higher is better.
cHR:       Cumulative Hit Rate; hit rate, confined to ratings above a certain threshold. Higher is better.
ARHR:      Average Reciprocal Hit Rank - Hit rate that takes the ranking into account. Higher is better.
Coverage:  Ratio of users for whom recommendations above a certain threshold exist. Higher is better.
Diversity: 1-S, where S is the average similarity score between every possible pair of recommendations
           for a given user. Higher means more diverse.
Novelty:   Average popularity rank of recommended items. Higher means more novel.
```

```
Using recommender  RBM
We recommend:
Toy Story (1995) 2.8893738
Dead Man Walking (1995) 2.8887234
James and the Giant Peach (1996) 2.8869526
Nixon (1995) 2.882759
Mr. Holland's Opus (1995) 2.8821468
Willy Wonka & the Chocolate Factory (1971) 2.8768373
Twelve Monkeys (a.k.a. 12 Monkeys) (1995) 2.8321264
Leaving Las Vegas (1995) 2.831291
Sense and Sensibility (1995) 2.8311553
Fargo (1996) 2.8257232

Using recommender  SVD++Untuned
We recommend:
Usual Suspects, The (1995) 5
Star Wars: Episode V - The Empire Strikes Back (1980) 5
Princess Bride, The (1987) 5
Goodfellas (1990) 5
Lock, Stock & Two Smoking Barrels (1998) 5
Shawshank Redemption, The (1994) 5
Patton (1970) 5
In the Name of the Father (1993) 5
Philadelphia Story, The (1940) 5
Singin' in the Rain (1952) 5

Using recommender  SVD Tuned
We recommend:
Usual Suspects, The (1995) 5
Pulp Fiction (1994) 5
Fight Club (1999) 5
Shawshank Redemption, The (1994) 5
Inside Job (2010) 5
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) 5
Godfather, The (1972) 5
General, The (1926) 5
Ran (1985) 5
Guess Who's Coming to Dinner (1967) 5

Using recommender  ContentKNN
We recommend:
Psycho (1960) 5
Troll 2 (1990) 5
Candyman: Farewell to the Flesh (1995) 5
2 Days in the Valley (1996) 5
Sherlock - A Study in Pink (2010) 5
Sound of Music, The (1965) 5
South Pacific (1958) 5
'Tis the Season for Love (2015) 5
Fog of War: Eleven Lessons from the Life of Robert S. McNamara, The (2003) 5
Message in a Bottle (1999) 5

Using recommender  Auto Rec
We recommend:
Silence of the Lambs, The (1991) 5
Princess Bride, The (1987) 5
Apocalypse Now (1979) 5
Back to the Future Part III (1990) 5
Texas Chainsaw Massacre, The (1974) 5
JFK (1991) 5
Unforgiven (1992) 5
This Is Spinal Tap (1984) 5
Waiting for Guffman (1996) 5
Pecker (1998) 5

Using recommender  Hybrid
We recommend:
Guess Who's Coming to Dinner (1967) 4.6381275333774425
Shawshank Redemption, The (1994) 4.589472211505933
Reign Over Me (2007) 4.552449596483468
Jules and Jim (Jules et Jim) (1961) 4.5504392218549965
Neon Genesis Evangelion: The End of Evangelion (Shin seiki Evangelion GekijÃ´-ban: Air/Magokoro wo, kimi ni) (1997) 4.546317577926959
Grave of the Fireflies (Hotaru no haka) (1988) 4.546125630098492
In the Name of the Father (1993) 4.541109563111314
Captain Fantastic (2016) 4.529957558072008
Mary and Max (2009) 4.527240995360906
Gladiator (1992) 4.525807842017585
```

Each of the base algorithms had pretty good movie recommendations. Unfortunately, it did not help with the hybrid model's algorithm.

***

**[Back to Top](#top)**

***

<a name="946"></a>
#### 9.4.6 - RBM | SVD++Untuned | SVDTuned = 33/34/33

In this model, I decided to try algorithms that gave me the best top ten recommendations for each one. I also used my tuned `RBM` model instead of the base version. I weighted each equally.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from RBMAlgorithm import RBMAlgorithm
from ContentKNNAlgorithm import ContentKNNAlgorithm
from AutoRecAlgorithm import AutoRecAlgorithm
from HybridAlgorithm import HybridAlgorithm
from surprise import SVD, SVDpp
from surprise import NormalPredictor
from Evaluator import Evaluator

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# RBM
RBM = RBMAlgorithm(epochs=40, hiddenDim=40, learningRate=0.2)

# SVD++ Untuned
SVDPlus = SVDpp()

# SVD Tuned
SVDTuned = SVD(n_epochs=11, lr_all=0.034, n_factors=90, reg_all=0.06)

# Random
Random = NormalPredictor()

#Content
#ContentKNN = ContentKNNAlgorithm()

#AutoRec
#{'hiddenDim': 200, 'learningRate': 0.1}
#AutoRec = AutoRecAlgorithm(hiddenDim=200, learningRate=0.1)

#Combine them
Hybrid = HybridAlgorithm([RBM, SVDPlus, SVDTuned], [0.33, 0.34, 0.33])

evaluator.AddAlgorithm(RBM, "RBM")
evaluator.AddAlgorithm(SVDPlus, "SVD++Untuned")
evaluator.AddAlgorithm(SVDTuned, "SVD Tuned")
#evaluator.AddAlgorithm(ContentKNN, "ContentKNN")
#evaluator.AddAlgorithm(AutoRec, "Auto Rec")
#evaluator.AddAlgorithm(Random, "Random")
evaluator.AddAlgorithm(Hybrid, "Hybrid")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)

Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  RBM ...
Evaluating accuracy...
Trained epoch  0
Trained epoch  1
Trained epoch  2
Trained epoch  3
Trained epoch  4
Trained epoch  5
Trained epoch  6
Trained epoch  7
Trained epoch  8
Trained epoch  9
Trained epoch  10
Trained epoch  11
Trained epoch  12
Trained epoch  13
Trained epoch  14
Trained epoch  15
Trained epoch  16
Trained epoch  17
Trained epoch  18
Trained epoch  19
Trained epoch  20
Trained epoch  21
Trained epoch  22
Trained epoch  23
Trained epoch  24
Trained epoch  25
Trained epoch  26
Trained epoch  27
Trained epoch  28
Trained epoch  29
Trained epoch  30
Trained epoch  31
Trained epoch  32
Trained epoch  33
Trained epoch  34
Trained epoch  35
Trained epoch  36
Trained epoch  37
Trained epoch  38
Trained epoch  39
Processing user  0
Processing user  50
Process

Below, the metrics for my Hybrid model are decent. It has one of the lowest `Novelty` scores I've seen yet. 
```
Algorithm    RMSE       MAE        HR         cHR        ARHR       Coverage   Diversity  Novelty   
RBM          1.2855     1.0927     0.0230     0.0230     0.0077     0.0000     0.0496     938.7416  
SVD++Untuned 0.8656     0.6650     0.0213     0.0213     0.0114     0.9164     0.0959     924.7326  
SVD Tuned    0.8686     0.6680     0.0180     0.0180     0.0080     0.9607     0.3015     2057.0823 
Hybrid       0.9218     0.7408     0.0311     0.0311     0.0100     0.4607     0.1731     1380.8639 

Legend:

RMSE:      Root Mean Squared Error. Lower values mean better accuracy.
MAE:       Mean Absolute Error. Lower values mean better accuracy.
HR:        Hit Rate; how often we are able to recommend a left-out rating. Higher is better.
cHR:       Cumulative Hit Rate; hit rate, confined to ratings above a certain threshold. Higher is better.
ARHR:      Average Reciprocal Hit Rank - Hit rate that takes the ranking into account. Higher is better.
Coverage:  Ratio of users for whom recommendations above a certain threshold exist. Higher is better.
Diversity: 1-S, where S is the average similarity score between every possible pair of recommendations
           for a given user. Higher means more diverse.
Novelty:   Average popularity rank of recommended items. Higher means more novel.
```

```
Using recommender  RBM
We recommend:
Willy Wonka & the Chocolate Factory (1971) 2.879406
Mr. Holland's Opus (1995) 2.8774168
James and the Giant Peach (1996) 2.875467
Nixon (1995) 2.8705785
Toy Story (1995) 2.8688858
Dead Man Walking (1995) 2.8661203
Independence Day (a.k.a. ID4) (1996) 2.8273323
Twister (1996) 2.8230257
Leaving Las Vegas (1995) 2.8192022
Fargo (1996) 2.8186142

Using recommender  SVD++Untuned
We recommend:
Usual Suspects, The (1995) 5
Star Wars: Episode V - The Empire Strikes Back (1980) 5
Princess Bride, The (1987) 5
Goodfellas (1990) 5
Lock, Stock & Two Smoking Barrels (1998) 5
Shawshank Redemption, The (1994) 5
Patton (1970) 5
In the Name of the Father (1993) 5
Philadelphia Story, The (1940) 5
Singin' in the Rain (1952) 5

Using recommender  SVD Tuned
We recommend:
Usual Suspects, The (1995) 5
Pulp Fiction (1994) 5
Fight Club (1999) 5
Shawshank Redemption, The (1994) 5
Inside Job (2010) 5
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) 5
Godfather, The (1972) 5
General, The (1926) 5
Ran (1985) 5
Guess Who's Coming to Dinner (1967) 5

Using recommender  Hybrid
We recommend:
Guess Who's Coming to Dinner (1967) 4.412769736976864
Shawshank Redemption, The (1994) 4.403609393458864
Usual Suspects, The (1995) 4.323466424084964
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) 4.31938546113286
Godfather, The (1972) 4.308208257264453
Streetcar Named Desire, A (1951) 4.302330330288717
Pulp Fiction (1994) 4.301286388541145
His Girl Friday (1940) 4.3010980533206045
Paths of Glory (1957) 4.298384399283833
Lawrence of Arabia (1962) 4.294990937981201
```

Unfortunately, the top ten recommendations for my Hybrid were not exciting, and not even on par with each of my component algorithms. I would not recommend this model.

***

**[Back to Top](#top)**

***

<a name="947"></a>
#### 9.4.7 - SVD++Untuned | SVDTuned = 50/50 *

Because I tended to heavily favor both SVD algorithms due to their movie recommendations, I thought to weigh them together in their own hybrid model. `SVD++Untuned` had a much better `Novelty` score while `SVDTuned` recommended less popular movies on average. However, I liked both their top 10 movie recommendations. I thought averaging them together would hopefully give me the best of each.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from RBMAlgorithm import RBMAlgorithm
from ContentKNNAlgorithm import ContentKNNAlgorithm
from AutoRecAlgorithm import AutoRecAlgorithm
from HybridAlgorithm import HybridAlgorithm
from surprise import SVD, SVDpp
from surprise import NormalPredictor
from Evaluator import Evaluator

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# RBM
RBM = RBMAlgorithm(epochs=40, hiddenDim=40, learningRate=0.2)

# SVD++ Untuned
SVDPlus = SVDpp()

# SVD Tuned
SVDTuned = SVD(n_epochs=11, lr_all=0.034, n_factors=90, reg_all=0.06)

Random = NormalPredictor()

#Content
#ContentKNN = ContentKNNAlgorithm()

#AutoRec
#{'hiddenDim': 200, 'learningRate': 0.1}
#AutoRec = AutoRecAlgorithm(hiddenDim=200, learningRate=0.1)

#Combine them
Hybrid = HybridAlgorithm([SVDPlus, SVDTuned], [0.5, 0.5])

#evaluator.AddAlgorithm(RBM, "RBM")
evaluator.AddAlgorithm(SVDPlus, "SVD++Untuned")
evaluator.AddAlgorithm(SVDTuned, "SVD Tuned")
#evaluator.AddAlgorithm(ContentKNN, "ContentKNN")
#evaluator.AddAlgorithm(AutoRec, "Auto Rec")
#evaluator.AddAlgorithm(Random, "Random")
evaluator.AddAlgorithm(Hybrid, "Hybrid")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)

Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  SVD++Untuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  SVD Tuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  Hybrid ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data

This is my first Hybrid Recommender that had a lower `RMSE` and higher Hit Rates than either of the component algorithms. Its `Novelty` score pretty good too.

```
Algorithm    RMSE       MAE        HR         cHR        ARHR       Coverage   Diversity  Novelty   
SVD++Untuned 0.8681     0.6674     0.0197     0.0197     0.0083     0.9246     0.0738     904.6954  
SVD Tuned    0.8655     0.6663     0.0230     0.0230     0.0090     0.9623     0.2877     2080.1600 
Hybrid       0.8577     0.6594     0.0328     0.0328     0.0137     0.9295     0.1716     1372.1110 

Legend:

RMSE:      Root Mean Squared Error. Lower values mean better accuracy.
MAE:       Mean Absolute Error. Lower values mean better accuracy.
HR:        Hit Rate; how often we are able to recommend a left-out rating. Higher is better.
cHR:       Cumulative Hit Rate; hit rate, confined to ratings above a certain threshold. Higher is better.
ARHR:      Average Reciprocal Hit Rank - Hit rate that takes the ranking into account. Higher is better.
Coverage:  Ratio of users for whom recommendations above a certain threshold exist. Higher is better.
Diversity: 1-S, where S is the average similarity score between every possible pair of recommendations
           for a given user. Higher means more diverse.
Novelty:   Average popularity rank of recommended items. Higher means more novel.
```

```


Using recommender  SVD++Untuned

Building recommendation model...
Computing recommendations...

We recommend:
Usual Suspects, The (1995) 5
Forrest Gump (1994) 5
Silence of the Lambs, The (1991) 5
Star Wars: Episode V - The Empire Strikes Back (1980) 5
Princess Bride, The (1987) 5
Goodfellas (1990) 5
Shining, The (1980) 5
American History X (1998) 5
Fight Club (1999) 5
Shawshank Redemption, The (1994) 5

Using recommender  SVD Tuned

Building recommendation model...
Computing recommendations...

We recommend:
Pulp Fiction (1994) 5
Silence of the Lambs, The (1991) 5
Fight Club (1999) 5
Shawshank Redemption, The (1994) 5
Departed, The (2006) 5
Life Is Beautiful (La Vita Ã¨ bella) (1997) 5
General, The (1926) 5
Witness for the Prosecution (1957) 5
Mary and Max (2009) 5
Ran (1985) 5

Using recommender  Hybrid

Building recommendation model...
Computing recommendations...

We recommend:
Usual Suspects, The (1995) 5
Pulp Fiction (1994) 5
Star Wars: Episode V - The Empire Strikes Back (1980) 5
Princess Bride, The (1987) 5
Fight Club (1999) 5
Shawshank Redemption, The (1994) 5
Departed, The (2006) 5
Philadelphia Story, The (1940) 5
Life Is Beautiful (La Vita Ã¨ bella) (1997) 5
In the Mood For Love (Fa yeung nin wa) (2000) 5
```

I'm a little disappointed that most of my movie recommendations are around 20 years old. However, my Hybrid model did recommend 7 movies that I actually like so I would definitely consider this one.

***

**[Back to Top](#top)**

***

<a name="948"></a>
#### 9.4.8 - SVD++Untuned | SVDTuned | Random = 33/34/33

My last model was one of my favorites, and I wanted to see if adding `Random` would make it even better. I like to add it `Random` because it could help recommend movies that a user would have never considered if it's out of his usual pattern.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from RBMAlgorithm import RBMAlgorithm
from ContentKNNAlgorithm import ContentKNNAlgorithm
from AutoRecAlgorithm import AutoRecAlgorithm
from HybridAlgorithm import HybridAlgorithm
from surprise import SVD, SVDpp
from surprise import NormalPredictor
from Evaluator import Evaluator

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# RBM
RBM = RBMAlgorithm(epochs=40, hiddenDim=40, learningRate=0.2)

# SVD++ Untuned
SVDPlus = SVDpp()

# SVD Tuned
SVDTuned = SVD(n_epochs=11, lr_all=0.034, n_factors=90, reg_all=0.06)

Random = NormalPredictor()

#Content
#ContentKNN = ContentKNNAlgorithm()

#AutoRec
#{'hiddenDim': 200, 'learningRate': 0.1}
#AutoRec = AutoRecAlgorithm(hiddenDim=200, learningRate=0.1)

#Combine them
Hybrid = HybridAlgorithm([SVDPlus, SVDTuned, Random], [0.33, 0.34, 0.33])

#evaluator.AddAlgorithm(RBM, "RBM")
evaluator.AddAlgorithm(SVDPlus, "SVD++Untuned")
evaluator.AddAlgorithm(SVDTuned, "SVD Tuned")
#evaluator.AddAlgorithm(ContentKNN, "ContentKNN")
#evaluator.AddAlgorithm(AutoRec, "Auto Rec")
evaluator.AddAlgorithm(Random, "Random")
evaluator.AddAlgorithm(Hybrid, "Hybrid")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)

Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  SVD++Untuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  SVD Tuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  Random ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data

Adding `Random` did worsen the metrics a little bit. `RMSE` is higher than without `Random` and the `Novelty` metric ballooned to almost 3000 as compared to 1380 in my last model.
```
Algorithm    RMSE       MAE        HR         cHR        ARHR       Coverage   Diversity  Novelty   
SVD++Untuned 0.8681     0.6674     0.0197     0.0197     0.0083     0.9246     0.0738     904.6954  
SVD Tuned    0.8655     0.6663     0.0230     0.0230     0.0090     0.9623     0.2877     2080.1600 
Random       1.4305     1.1461     0.0115     0.0115     0.0041     1.0000     0.0540     860.3918  
Hybrid       0.9470     0.7396     0.0098     0.0098     0.0032     0.9951     0.4422     2997.0620 

Legend:

RMSE:      Root Mean Squared Error. Lower values mean better accuracy.
MAE:       Mean Absolute Error. Lower values mean better accuracy.
HR:        Hit Rate; how often we are able to recommend a left-out rating. Higher is better.
cHR:       Cumulative Hit Rate; hit rate, confined to ratings above a certain threshold. Higher is better.
ARHR:      Average Reciprocal Hit Rank - Hit rate that takes the ranking into account. Higher is better.
Coverage:  Ratio of users for whom recommendations above a certain threshold exist. Higher is better.
Diversity: 1-S, where S is the average similarity score between every possible pair of recommendations
           for a given user. Higher means more diverse.
Novelty:   Average popularity rank of recommended items. Higher means more novel.
```

```
Using recommender  SVD++Untuned

Building recommendation model...
Computing recommendations...

We recommend:
Usual Suspects, The (1995) 5
Pulp Fiction (1994) 5
Reservoir Dogs (1992) 5
Monty Python and the Holy Grail (1975) 5
Princess Bride, The (1987) 5
Goodfellas (1990) 5
Shining, The (1980) 5
Lock, Stock & Two Smoking Barrels (1998) 5
Fight Club (1999) 5
Shawshank Redemption, The (1994) 5

Using recommender  SVD Tuned

Building recommendation model...
Computing recommendations...

We recommend:
Pulp Fiction (1994) 5
Fight Club (1999) 5
Shawshank Redemption, The (1994) 5
Departed, The (2006) 5
Crumb (1994) 5
In the Name of the Father (1993) 5
Philadelphia Story, The (1940) 5
Rear Window (1954) 5
Beautiful Thing (1996) 5
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) 5

Using recommender  Random

Building recommendation model...
Computing recommendations...

We recommend:
Tommy Boy (1995) 5
James and the Giant Peach (1996) 5
Terminator, The (1984) 5
Highlander (1986) 5
Young Frankenstein (1974) 5
Best Men (1997) 5
All Quiet on the Western Front (1930) 5
Psycho (1998) 5
Rushmore (1998) 5
Thin Red Line, The (1998) 5

Using recommender  Hybrid

Building recommendation model...
Computing recommendations...

We recommend:
Silence of the Lambs, The (1991) 5
Star Wars: Episode V - The Empire Strikes Back (1980) 5
Apocalypse Now (1979) 5
Rear Window (1954) 5
Crimes and Misdemeanors (1989) 5
O Brother, Where Art Thou? (2000) 5
Thirteen Days (2000) 5
Under the Sand (2000) 5
White Squall (1996) 5
Fly Away Home (1996) 5
```

The recommendations are not the best. I probably only like three or four of them. I would not recommend this model unless I had no other choice.

***

**[Back to Top](#top)**

***

<a name="948"></a>
#### 9.4.08 - SVD++Untuned | SVDTuned | Random = 33/34/33

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from RBMAlgorithm import RBMAlgorithm
from ContentKNNAlgorithm import ContentKNNAlgorithm
from AutoRecAlgorithm import AutoRecAlgorithm
from HybridAlgorithm import HybridAlgorithm
from surprise import SVD, SVDpp
from surprise import NormalPredictor
from Evaluator import Evaluator

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# RBM
RBM = RBMAlgorithm(epochs=40, hiddenDim=40, learningRate=0.2)

# SVD++ Untuned
SVDPlus = SVDpp()

# SVD Tuned
SVDTuned = SVD(n_epochs=11, lr_all=0.034, n_factors=90, reg_all=0.06)

Random = NormalPredictor()

#Content
#ContentKNN = ContentKNNAlgorithm()

#AutoRec
#{'hiddenDim': 200, 'learningRate': 0.1}
AutoRec = AutoRecAlgorithm(hiddenDim=200, learningRate=0.1)

#Combine them
Hybrid = HybridAlgorithm([RBM, AutoRec, Random], [0.34, 0.33, 0.33])

evaluator.AddAlgorithm(RBM, "RBM")
#evaluator.AddAlgorithm(SVDPlus, "SVD++Untuned")
#evaluator.AddAlgorithm(SVDTuned, "SVD Tuned")
#evaluator.AddAlgorithm(ContentKNN, "ContentKNN")
evaluator.AddAlgorithm(AutoRec, "Auto Rec")
evaluator.AddAlgorithm(Random, "Random")
evaluator.AddAlgorithm(Hybrid, "Hybrid")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)

Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  RBM ...
Evaluating accuracy...
Trained epoch  0
Trained epoch  1
Trained epoch  2
Trained epoch  3
Trained epoch  4
Trained epoch  5
Trained epoch  6
Trained epoch  7
Trained epoch  8
Trained epoch  9
Trained epoch  10
Trained epoch  11
Trained epoch  12
Trained epoch  13
Trained epoch  14
Trained epoch  15
Trained epoch  16
Trained epoch  17
Trained epoch  18
Trained epoch  19
Trained epoch  20
Trained epoch  21
Trained epoch  22
Trained epoch  23
Trained epoch  24
Trained epoch  25
Trained epoch  26
Trained epoch  27
Trained epoch  28
Trained epoch  29
Trained epoch  30
Trained epoch  31
Trained epoch  32
Trained epoch  33
Trained epoch  34
Trained epoch  35
Trained epoch  36
Trained epoch  37
Trained epoch  38
Trained epoch  39
Processing user  0
Processing user  50
Process

***

**[Back to Top](#top)**

***

<a name="948"></a>
#### 9.4.8 - SVD++Untuned | SVDTuned | Random = 33/34/33

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from RBMAlgorithm import RBMAlgorithm
from ContentKNNAlgorithm import ContentKNNAlgorithm
from AutoRecAlgorithm import AutoRecAlgorithm
from HybridAlgorithm import HybridAlgorithm
from surprise import SVD, SVDpp
from surprise import NormalPredictor
from Evaluator import Evaluator

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# RBM
RBM = RBMAlgorithm(epochs=40, hiddenDim=40, learningRate=0.2)

# SVD++ Untuned
SVDPlus = SVDpp()

# SVD Tuned
SVDTuned = SVD(n_epochs=11, lr_all=0.034, n_factors=90, reg_all=0.06)

Random = NormalPredictor()

#Content
ContentKNN = ContentKNNAlgorithm()

#AutoRec
#{'hiddenDim': 200, 'learningRate': 0.1}
AutoRec = AutoRecAlgorithm(hiddenDim=200, learningRate=0.1)

#Combine them
Hybrid = HybridAlgorithm([RBM, SVDPlus, SVDTuned, ContentKNN, AutoRec, Random], [0.1, 0.1, 0.1, 0.1, 0.1, 0.5])

evaluator.AddAlgorithm(RBM, "RBM")
evaluator.AddAlgorithm(SVDPlus, "SVD++Untuned")
evaluator.AddAlgorithm(SVDTuned, "SVD Tuned")
evaluator.AddAlgorithm(ContentKNN, "ContentKNN")
evaluator.AddAlgorithm(AutoRec, "Auto Rec")
evaluator.AddAlgorithm(Random, "Random")
evaluator.AddAlgorithm(Hybrid, "Hybrid")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)

Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  RBM ...
Evaluating accuracy...
Trained epoch  0
Trained epoch  1
Trained epoch  2
Trained epoch  3
Trained epoch  4
Trained epoch  5
Trained epoch  6
Trained epoch  7
Trained epoch  8
Trained epoch  9
Trained epoch  10
Trained epoch  11
Trained epoch  12
Trained epoch  13
Trained epoch  14
Trained epoch  15
Trained epoch  16
Trained epoch  17
Trained epoch  18
Trained epoch  19
Trained epoch  20
Trained epoch  21
Trained epoch  22
Trained epoch  23
Trained epoch  24
Trained epoch  25
Trained epoch  26
Trained epoch  27
Trained epoch  28
Trained epoch  29
Trained epoch  30
Trained epoch  31
Trained epoch  32
Trained epoch  33
Trained epoch  34
Trained epoch  35
Trained epoch  36
Trained epoch  37
Trained epoch  38
Trained epoch  39
Processing user  0
Processing user  50
Process

***

**[Back to Top](#top)**

***

<a name="948"></a>
#### 9.4.8 - SVD++Untuned | SVDTuned | Random = 33/34/33

Content: Date Only but changed knn algoritm to similarity 4.5

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Mar 18 17:32:44 2021

@author: Maribel
"""

# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from ContentKNNAlgorithm import ContentKNNAlgorithm
from HybridAlgorithm import HybridAlgorithm
from Evaluator import Evaluator
from surprise import NormalPredictor

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# Just make random recommendations
Random = NormalPredictor()
evaluator.AddAlgorithm(Random, "Random")

#Content
ContentKNN = ContentKNNAlgorithm()

#Combine them
Hybrid = HybridAlgorithm([ContentKNN, Random], [0.9, 0.1])

evaluator.AddAlgorithm(ContentKNN, "ContentKNN")
evaluator.AddAlgorithm(Hybrid, "Hybrid")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)


Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  Random ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  ContentKNN ...
Evaluating accuracy...
Computing content-based similarity matrix...
0  of  8805
100  of  8805
200  of  8805
300  of  8805
400  of  8805
500  of  8805
600  of  8805
700  of  8805
800  of  8805
900  of  8805
1000  of  8805
1100  of  8805
1200  of  8805
1300  of  8805
1400  of  8805
1500  of  8805
1600  of  8805
1700  of  8805
1800  of  8805
1900  of  8805
2000  of  8805
2100  of  8805
2200  of  8805
2300  of  8805
2400  of  8805
2500  of  8805
2600  of  880

***

**[Back to Top](#top)**

***

<a name="948"></a>
#### 9.4.8 - SVD++Untuned | SVDTuned | Random = 33/34/33

Similarity score 4.5

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from RBMAlgorithm import RBMAlgorithm
from ContentKNNAlgorithm import ContentKNNAlgorithm
from AutoRecAlgorithm import AutoRecAlgorithm
from HybridAlgorithm import HybridAlgorithm
from surprise import SVD, SVDpp
from surprise import NormalPredictor
from Evaluator import Evaluator

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# RBM
RBM = RBMAlgorithm(epochs=40, hiddenDim=40, learningRate=0.2)

# SVD++ Untuned
SVDPlus = SVDpp()

# SVD Tuned
SVDTuned = SVD(n_epochs=11, lr_all=0.034, n_factors=90, reg_all=0.06)

Random = NormalPredictor()

#Content
#ContentKNN = ContentKNNAlgorithm()

#AutoRec
#{'hiddenDim': 200, 'learningRate': 0.1}
AutoRec = AutoRecAlgorithm(hiddenDim=200, learningRate=0.1)

#Combine them
Hybrid = HybridAlgorithm([RBM, SVDTuned, ContentKNN, Random], [0.25, 0.25, 0.25, 0.25])

evaluator.AddAlgorithm(RBM, "RBM")
#evaluator.AddAlgorithm(SVDPlus, "SVD++Untuned")
evaluator.AddAlgorithm(SVDTuned, "SVD Tuned")
evaluator.AddAlgorithm(ContentKNN, "ContentKNN")
#evaluator.AddAlgorithm(AutoRec, "Auto Rec")
evaluator.AddAlgorithm(Random, "Random")
evaluator.AddAlgorithm(Hybrid, "Hybrid")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)

Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  Random ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  ContentKNN ...
Evaluating accuracy...
Computing content-based similarity matrix...
0  of  8805
100  of  8805
200  of  8805
300  of  8805
400  of  8805
500  of  8805
600  of  8805
700  of  8805
800  of  8805
900  of  8805
1000  of  8805
1100  of  8805
1200  of  8805
1300  of  8805
1400  of  8805
1500  of  8805
1600  of  8805
1700  of  8805
1800  of  8805
1900  of  8805
2000  of  8805
2100  of  8805
2200  of  8805
2300  of  8805
2400  of  8805
2500  of  8805
2600  of  880

***

**[Back to Top](#top)**

***

<a name="948"></a>
#### 9.4.8 - SVD++Untuned | SVDTuned | Random = 33/34/33

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from RBMAlgorithm import RBMAlgorithm
from ContentKNNAlgorithm import ContentKNNAlgorithm
from AutoRecAlgorithm import AutoRecAlgorithm
from HybridAlgorithm import HybridAlgorithm
from surprise import SVD, SVDpp
from surprise import NormalPredictor
from Evaluator import Evaluator

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# RBM
RBM = RBMAlgorithm(epochs=40, hiddenDim=40, learningRate=0.2)

# SVD++ Untuned
SVDPlus = SVDpp()

# SVD Untuned
SVDUntuned = SVD()

# SVD Tuned
SVDTuned = SVD(n_epochs=11, lr_all=0.034, n_factors=90, reg_all=0.06)

Random = NormalPredictor()

#Content
ContentKNN = ContentKNNAlgorithm()

#AutoRec
#{'hiddenDim': 200, 'learningRate': 0.1}
AutoRec = AutoRecAlgorithm(hiddenDim=200, learningRate=0.1)

#Combine them
Hybrid = HybridAlgorithm([SVDTuned, SVDUntuned], [0.5, 0.5])

#evaluator.AddAlgorithm(RBM, "RBM")
#evaluator.AddAlgorithm(SVDPlus, "SVD++Untuned")
evaluator.AddAlgorithm(SVDUntuned, "SVD Untuned")
evaluator.AddAlgorithm(SVDTuned, "SVD Tuned")
#evaluator.AddAlgorithm(ContentKNN, "ContentKNN")
#evaluator.AddAlgorithm(AutoRec, "Auto Rec")
#evaluator.AddAlgorithm(Random, "Random")
evaluator.AddAlgorithm(Hybrid, "Hybrid")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)

Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  SVD Untuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  SVD Tuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  Hybrid ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data 

***

**[Back to Top](#top)**

***

<a name="948"></a>
#### 9.4.8 - SVD++Untuned | SVDTuned | Random = 33/34/33

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from RBMAlgorithm import RBMAlgorithm
from ContentKNNAlgorithm import ContentKNNAlgorithm
from AutoRecAlgorithm import AutoRecAlgorithm
from HybridAlgorithm import HybridAlgorithm
from surprise import SVD, SVDpp
from surprise import NormalPredictor
from Evaluator import Evaluator

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# RBM
RBM = RBMAlgorithm(epochs=40, hiddenDim=40, learningRate=0.2)

# SVD++ Untuned
SVDPlus = SVDpp()

# SVD Untuned
SVDUntuned = SVD()

# SVD Tuned
SVDTuned = SVD(n_epochs=11, lr_all=0.034, n_factors=90, reg_all=0.06)

Random = NormalPredictor()

#Content
ContentKNN = ContentKNNAlgorithm()

#AutoRec
#{'hiddenDim': 200, 'learningRate': 0.1}
AutoRec = AutoRecAlgorithm(hiddenDim=200, learningRate=0.1)

#Combine them
Hybrid = HybridAlgorithm([SVDTuned, SVDUntuned, Random], [0.34, 0.33, 0.33])

#evaluator.AddAlgorithm(RBM, "RBM")
#evaluator.AddAlgorithm(SVDPlus, "SVD++Untuned")
evaluator.AddAlgorithm(SVDUntuned, "SVD Untuned")
evaluator.AddAlgorithm(SVDTuned, "SVD Tuned")
#evaluator.AddAlgorithm(ContentKNN, "ContentKNN")
#evaluator.AddAlgorithm(AutoRec, "Auto Rec")
evaluator.AddAlgorithm(Random, "Random")
evaluator.AddAlgorithm(Hybrid, "Hybrid")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)

Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  SVD Untuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  SVD Tuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  Hybrid ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data 

***

**[Back to Top](#top)**

***

<a name="948"></a>
#### 9.4.8 - SVD++Untuned | SVDTuned | Random = 33/34/33

content over 4.5 and it's both genre and years

update: shit I didn't add random

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from RBMAlgorithm import RBMAlgorithm
from ContentKNNAlgorithm import ContentKNNAlgorithm
from AutoRecAlgorithm import AutoRecAlgorithm
from HybridAlgorithm import HybridAlgorithm
from surprise import SVD, SVDpp
from surprise import NormalPredictor
from Evaluator import Evaluator

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# RBM
RBM = RBMAlgorithm(epochs=40, hiddenDim=40, learningRate=0.2)

# SVD++ Untuned
SVDPlus = SVDpp()

# SVD Tuned
SVDTuned = SVD(n_epochs=11, lr_all=0.034, n_factors=90, reg_all=0.06)

Random = NormalPredictor()

#Content
ContentKNN = ContentKNNAlgorithm()

#AutoRec
#{'hiddenDim': 200, 'learningRate': 0.1}
AutoRec = AutoRecAlgorithm(hiddenDim=200, learningRate=0.1)

#Combine them
Hybrid = HybridAlgorithm([SVDPlus, SVDTuned, ContentKNN, Random], [0.34, 0.33, 0.33])

#evaluator.AddAlgorithm(RBM, "RBM")
evaluator.AddAlgorithm(SVDPlus, "SVD++Untuned")
evaluator.AddAlgorithm(SVDTuned, "SVD Tuned")
evaluator.AddAlgorithm(ContentKNN, "ContentKNN")
#evaluator.AddAlgorithm(AutoRec, "Auto Rec")
#evaluator.AddAlgorithm(Random, "Random")
evaluator.AddAlgorithm(Hybrid, "Hybrid")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)

Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  SVD++Untuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  SVD Tuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  ContentKNN ...
Evaluating accuracy...
Computing content-based similarity matrix...
0  of  8805
100  of  8805
200  of  8805
300  of  8805
400  of  8805
5

***

**[Back to Top](#top)**

***

<a name="948"></a>
#### 9.4.8 - SVD++Untuned | SVDTuned | Random = 33/34/33

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from RBMAlgorithm import RBMAlgorithm
from ContentKNNAlgorithm import ContentKNNAlgorithm
from AutoRecAlgorithm import AutoRecAlgorithm
from HybridAlgorithm import HybridAlgorithm
from surprise import SVD, SVDpp
from surprise import NormalPredictor
from Evaluator import Evaluator

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# RBM
RBM = RBMAlgorithm(epochs=40, hiddenDim=40, learningRate=0.2)

# SVD++ Untuned
SVDPlus = SVDpp()

# SVD Untuned
SVDUntuned = SVD()

# SVD Tuned
SVDTuned = SVD(n_epochs=11, lr_all=0.034, n_factors=90, reg_all=0.06)

Random = NormalPredictor()

#Content
ContentKNN = ContentKNNAlgorithm()

#AutoRec
#{'hiddenDim': 200, 'learningRate': 0.1}
AutoRec = AutoRecAlgorithm(hiddenDim=200, learningRate=0.1)

#Combine them
Hybrid = HybridAlgorithm([RBM, SVDUntuned, SVDTuned, ContentKNN], [0.25, 0.25, 0.25, 0.25])

evaluator.AddAlgorithm(RBM, "RBM")
#evaluator.AddAlgorithm(SVDPlus, "SVD++Untuned")
evaluator.AddAlgorithm(SVDUntuned, "SVD Untuned")
evaluator.AddAlgorithm(SVDTuned, "SVD Tuned")
evaluator.AddAlgorithm(ContentKNN, "ContentKNN")
#evaluator.AddAlgorithm(AutoRec, "Auto Rec")
#evaluator.AddAlgorithm(Random, "Random")
evaluator.AddAlgorithm(Hybrid, "Hybrid")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)

Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  SVD++Untuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  SVD Tuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  ContentKNN ...
Evaluating accuracy...
Computing content-based similarity matrix...
0  of  8805
100  of  8805
200  of  8805
300  of  8805
400  of  8805
5

***

**[Back to Top](#top)**

***

<a name="948"></a>
#### 9.4.8 - SVD++Untuned | SVDTuned | Random = 33/34/33

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from RBMAlgorithm import RBMAlgorithm
from ContentKNNAlgorithm import ContentKNNAlgorithm
from AutoRecAlgorithm import AutoRecAlgorithm
from HybridAlgorithm import HybridAlgorithm
from surprise import SVD, SVDpp
from surprise import NormalPredictor
from Evaluator import Evaluator

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# RBM
RBM = RBMAlgorithm(epochs=40, hiddenDim=40, learningRate=0.2)

# SVD++ Untuned
SVDPlus = SVDpp()

# SVD Untuned
SVDUntuned = SVD()

# SVD Tuned
SVDTuned = SVD(n_epochs=11, lr_all=0.034, n_factors=90, reg_all=0.06)

Random = NormalPredictor()

#Content
ContentKNN = ContentKNNAlgorithm()

#AutoRec
#{'hiddenDim': 200, 'learningRate': 0.1}
AutoRec = AutoRecAlgorithm(hiddenDim=200, learningRate=0.1)

#Combine them
Hybrid = HybridAlgorithm([RBM, SVDPlus, SVDTuned, Random], [0.25, 0.25, 0.25, 0.25])

evaluator.AddAlgorithm(RBM, "RBM")
evaluator.AddAlgorithm(SVDPlus, "SVD++Untuned")
evaluator.AddAlgorithm(SVDTuned, "SVD Tuned")
#evaluator.AddAlgorithm(ContentKNN, "ContentKNN")
#evaluator.AddAlgorithm(AutoRec, "Auto Rec")
evaluator.AddAlgorithm(Random, "Random")
evaluator.AddAlgorithm(Hybrid, "Hybrid")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)

Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  SVD++Untuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  SVD Tuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  ContentKNN ...
Evaluating accuracy...
Computing content-based similarity matrix...
0  of  8805
100  of  8805
200  of  8805
300  of  8805
400  of  8805
5

***

**[Back to Top](#top)**

***

<a name="948"></a>
#### 9.4.8 - SVD++Untuned | SVDTuned | Random = 33/34/33

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu May  3 11:11:13 2018

@author: Frank
"""

from MovieLens import MovieLens
from RBMAlgorithm import RBMAlgorithm
from ContentKNNAlgorithm import ContentKNNAlgorithm
from AutoRecAlgorithm import AutoRecAlgorithm
from HybridAlgorithm import HybridAlgorithm
from surprise import SVD, SVDpp
from surprise import NormalPredictor
from Evaluator import Evaluator

import random
import numpy as np

def LoadMovieLensData():
    ml = MovieLens()
    print("Loading movie ratings...")
    data = ml.loadMovieLensLatestSmall()
    print("\nComputing movie popularity ranks so we can measure novelty later...")
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

np.random.seed(29)
random.seed(29)

# Load up common data set for the recommender algorithms
(ml, evaluationData, rankings) = LoadMovieLensData()

# Construct an Evaluator to, you know, evaluate them
evaluator = Evaluator(evaluationData, rankings)

# RBM
RBM = RBMAlgorithm(epochs=40, hiddenDim=40, learningRate=0.2)

# SVD++ Untuned
SVDPlusUntuned = SVDpp()

# SVD++ Tuned
SVDPlusTuned = SVDpp(n_epochs=11, lr_all=0.034, n_factors=90, reg_all=0.06)

# SVD Untuned
SVDUntuned = SVD()

# SVD Tuned
SVDTuned = SVD(n_epochs=11, lr_all=0.034, n_factors=90, reg_all=0.06)

Random = NormalPredictor()

#Content
ContentKNN = ContentKNNAlgorithm()

#AutoRec
#{'hiddenDim': 200, 'learningRate': 0.1}
AutoRec = AutoRecAlgorithm(hiddenDim=200, learningRate=0.1)

#Combine them
Hybrid = HybridAlgorithm([SVDPlusUntuned, SVDPlusTuned, SVDUntuned, SVDTuned], [0.25, 0.25, 0.25, 0.25])

#evaluator.AddAlgorithm(RBM, "RBM")
evaluator.AddAlgorithm(SVDPlusUntuned, "SVD++Untuned")
evaluator.AddAlgorithm(SVDPlusTuned, "SVD++Tuned")
evaluator.AddAlgorithm(SVDUntuned, "SVD Untuned")
evaluator.AddAlgorithm(SVDTuned, "SVD Tuned")
#evaluator.AddAlgorithm(ContentKNN, "ContentKNN")
#evaluator.AddAlgorithm(AutoRec, "Auto Rec")
evaluator.AddAlgorithm(Random, "Random")
evaluator.AddAlgorithm(Hybrid, "Hybrid")

# Fight!
evaluator.Evaluate(True)

evaluator.SampleTopNRecs(ml)

Loading movie ratings...

Computing movie popularity ranks so we can measure novelty later...
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating  SVD++Untuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  SVD++Tuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with full data set...
Analyzing coverage, diversity, and novelty...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Analysis complete.
Evaluating  SVD Untuned ...
Evaluating accuracy...
Evaluating top-N with leave-one-out...
Computing hit-rate and rank metrics...
Computing recommendations with ful

***

**[Back to Top](#top)**

***

<a name="948"></a>
#### 9.4.8 - SVD++Untuned | SVDTuned | Random = 33/34/33

A good hybrid recommender won't just average out the performance of each one. It will leverage the strengths of some recommenders to augment the weaknesses of others, and leave you with a system that's more than just the sum of its parts. Developing recommender systems sometimes feels like more of an art than a science, and choosing how to combine different algorithms can feel like choosing different colors in a painting that complement each other. 